In [34]:
import pandas as pd
import sqlite3 
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from string import punctuation 
import string
import numpy as np
import re

# Importing datasets from SQLite

In [35]:
# INSERT YOUR DATABASE NAME HERE
conn = sqlite3.connect('<YOUR DATABASE>')
cur = conn.cursor()

In [36]:
dfusers=pd.read_sql_query('''SELECT id as users_id, username, doctor, age, sex, member_since, location FROM Users''', conn)
dfusers.head()

,users_id,username,doctor,age,sex,member_since,location
0,1,Worm99,NaN,NaN,None,2007-01-01 00:00:00,None
1,2,Forum-OD-MP,NaN,NaN,None,2005-11-01 00:00:00,None
2,3,krysejens,NaN,NaN,None,2009-11-01 00:00:00,None
3,4,"Sandra M Brown, MD",1.0,NaN,Female,None,"Concord, NC"
4,5,Sigris,NaN,NaN,None,2010-03-01 00:00:00,None


In [37]:
# Total number of users
len(dfusers)

99

In [38]:
# CHOOSE 1 FOR DOCTORS; 0 FOR PATIENTS
dfptusers=dfusers[dfusers['doctor']==1]
dfptusers.head()

,users_id,username,doctor,age,sex,member_since,location
3,4,"Sandra M Brown, MD",1.0,NaN,Female,None,"Concord, NC"
10,11,"Christopher C Shen, MD",1.0,NaN,Male,None,"Las Vegas, NV"
13,14,"John C Hagan III, MD, FACS, FAAO",1.0,NaN,Male,None,"Kansas City, MO"
15,16,"Sandy Feldman, MD, MS",1.0,NaN,Female,None,"San Diego, CA"
17,18,"Michael J Kutryb, MD",1.0,NaN,Male,None,"Titusville, FL"


In [39]:
# Total number of patients or doctors
len(dfptusers)

10

In [40]:
# Get summary of all posts
dfposts=pd.read_sql_query('''SELECT Posts.id, Posts.threads_id, Posts.users_id, Posts.number, Posts.content, Posts.date, Posts.sent_score, Posts.label FROM Posts INNER JOIN Threads ON (Posts.threads_id = threads.id)''', conn)
dfposts.head()

,id,threads_id,users_id,number,content,date,sent_score,label
0,1,1,1,0,"\n Hi everyone,\r I suffer from...",2007-01-16 00:00:00,-0.364303,negative
1,2,1,2,1,\n you need a specialist. probably...,2007-01-16 00:00:00,-0.880506,negative
2,3,2,3,0,\n I am new to Las Vegas and am...,2009-11-11 15:51:35,0.593613,positive
3,4,2,4,1,\n Go to www.aao.org and search for...,2009-11-11 19:42:07,0.000000,neutral
4,5,3,5,0,\n My sons right eye has recede...,2010-03-20 17:41:13,-0.677034,negative


In [41]:
# Total number of posts
len(dfposts)

223

In [42]:
# Posts from patients or doctors
dfptposts=pd.merge(left=dfptusers[['users_id','username']], right=dfposts, how='inner', left_on='users_id', right_on='users_id')
dfptposts.head(50)

,users_id,username,id,threads_id,number,content,date,sent_score,label
0,4,"Sandra M Brown, MD",4,2,1,\n Go to www.aao.org and search for...,2009-11-11 19:42:07,0.000000,neutral
1,4,"Sandra M Brown, MD",6,3,1,\n Your son should be evaluated by ...,2010-03-30 17:07:23,0.860021,positive
2,4,"Sandra M Brown, MD",8,4,1,\n Is it on your eyeBALL or on your...,2010-01-28 08:17:25,-0.864718,negative
3,4,"Sandra M Brown, MD",10,5,1,\n You have ptosis. Obviously ptosi...,2009-11-05 17:03:54,-0.760373,negative
4,4,"Sandra M Brown, MD",12,6,1,\n It sounds like the muscle that m...,2010-09-27 15:34:02,-0.323683,negative
5,4,"Sandra M Brown, MD",15,7,1,\n Ptosis can be corrected with sur...,2009-11-05 14:30:35,0.000000,neutral
6,4,"Sandra M Brown, MD",26,12,1,\n This sounds like congenital obst...,2010-06-23 19:42:02,-0.291884,negative
7,4,"Sandra M Brown, MD",32,15,1,\n Your eyeball is not smaller if i...,2010-08-15 08:40:17,-0.685404,negative
8,4,"Sandra M Brown, MD",55,23,1,\n Your question cannot be answered...,2010-06-16 21:43:59,-0.910491,negative
9,4,"Sandra M Brown, MD",63,27,1,\n I think at this point you need a...,2010-09-27 16:40:38,0.700008,positive


In [43]:
# Total posts from patients or doctors
len(dfptposts)

77

In [44]:
# Get keyword scores
dftargetedsent=pd.read_sql_query('''SELECT * FROM KeywordsScores''', conn)
dftargetedsent.head(15)

,posts_id,keywords_id,sent_score,relevance,sadness_score,joy_score,fear_score,disgust_score,anger_score
0,1,1,-0.504713,0.645569,0.341329,0.071254,0.316608,0.232325,0.003045
1,1,2,0.000000,0.638464,0.159982,0.529674,0.094432,0.052398,0.026757
2,1,3,0.000000,0.629403,0.263028,0.275554,0.262074,0.081493,0.048659
3,1,4,0.965996,0.603728,0.279632,0.114120,0.182594,0.116771,0.051658
4,1,5,0.965996,0.573643,0.167237,0.594232,0.061398,0.018594,0.017416
5,1,6,-0.796117,0.571338,0.167237,0.594232,0.061398,0.018594,0.017416
6,1,7,0.000000,0.561941,0.174765,0.065790,0.184985,0.125459,0.137143
7,1,8,-0.796117,0.561113,0.577171,0.515473,0.264536,0.077507,0.068880
8,1,9,-0.504713,0.559001,0.224792,0.063617,0.206331,0.024208,0.066890
9,1,10,-0.504713,0.540631,0.224792,0.063617,0.206331,0.024208,0.066890


In [45]:
# Add labels
conditions = [
    (dftargetedsent['sent_score'] == 0),
    (dftargetedsent['sent_score'] > 0 ),
    (dftargetedsent['sent_score'] < 0 )]
choices = ['neutral', 'positive', 'negative']
dftargetedsent['label'] = np.select(conditions, choices, default=' ')

In [46]:
dftargetedsent.head(20)

,posts_id,keywords_id,sent_score,relevance,sadness_score,joy_score,fear_score,disgust_score,anger_score,label
0,1,1,-0.504713,0.645569,0.341329,0.071254,0.316608,0.232325,0.003045,negative
1,1,2,0.000000,0.638464,0.159982,0.529674,0.094432,0.052398,0.026757,neutral
2,1,3,0.000000,0.629403,0.263028,0.275554,0.262074,0.081493,0.048659,neutral
3,1,4,0.965996,0.603728,0.279632,0.114120,0.182594,0.116771,0.051658,positive
4,1,5,0.965996,0.573643,0.167237,0.594232,0.061398,0.018594,0.017416,positive
5,1,6,-0.796117,0.571338,0.167237,0.594232,0.061398,0.018594,0.017416,negative
6,1,7,0.000000,0.561941,0.174765,0.065790,0.184985,0.125459,0.137143,neutral
7,1,8,-0.796117,0.561113,0.577171,0.515473,0.264536,0.077507,0.068880,negative
8,1,9,-0.504713,0.559001,0.224792,0.063617,0.206331,0.024208,0.066890,negative
9,1,10,-0.504713,0.540631,0.224792,0.063617,0.206331,0.024208,0.066890,negative


# Process keywords

In [47]:
dfkeywords=pd.read_sql_query('SELECT * FROM Keywords', conn)
display(dfkeywords.head(10));

,id,name
0,1,type of surgery
1,2,regular eye doctor
2,3,eye doctor
3,4,aesthetic appearance of the eye
4,5,vision
5,6,shrunken eye
6,7,scleral shells
7,8,eye
8,9,issue
9,10,patent


In [48]:
# Remove punctuation 
exclude = set(string.punctuation)

def remove_punctuation(x):
    try:
        x = ''.join(ch for ch in x if ch not in exclude)
    except:
        pass
    return x

dfkeywords['kwlower']=dfkeywords['name'].apply(lambda x: x.lower())
dfkeywords['kwnopunc']=dfkeywords['kwlower'].apply(remove_punctuation)

In [49]:
dfkeywords['kwtoken']=dfkeywords['kwnopunc'].apply(lambda x: word_tokenize(x))
stop_words=set(stopwords.words('english')) 
dfkeywords['kwnostop']=dfkeywords['kwtoken'].apply(lambda x: [w for w in x if not w in stop_words])
lemmatizer  = WordNetLemmatizer() 
dfkeywords['kwlemma']=dfkeywords['kwnostop'].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

In [50]:
# Keyword transformation
import re

# This example function would transform keywords containing all regular 
# expressions (*lid*, *bleph* and *eye) to a new keyword "blepharoplasty eyelid"
def kwclean(lemmalist):
    res = None
    if all(list(filter(reg.match, lemmalist)) for reg in [re.compile('.*lid.*'), re.compile('bleph.*'), re.compile('eye.*')]):
        res = ['blepharoplasty', 'eyelid']
        
    return res or lemmalist

In [51]:
# Example of transformation 
kwclean(['eye lid', 'blephariplasty'])

['blepharoplasty', 'eyelid']

In [52]:
dfkeywords['kwcleaned']=dfkeywords['kwlemma'].apply(lambda x: kwclean(x))
dfkeywords['kwclean']=dfkeywords['kwcleaned'].apply(lambda x: ' '.join(x))

In [53]:
dfkeywords=dfkeywords[['id', 'name','kwclean']]
dfkwclean=dfkeywords[['id', 'kwclean']]

# Keyword aggregation for chosen group (patients or doctors)

In [54]:
# Merge keywords with sentiment scores
dfkwsent=pd.merge(left=dftargetedsent, right=dfkwclean, how='left', left_on='keywords_id', right_on='id')
dfkwsent=dfkwsent.drop('id', axis=1)

In [55]:
dfkwsent.head()

,posts_id,keywords_id,sent_score,relevance,sadness_score,joy_score,fear_score,disgust_score,anger_score,label,kwclean
0,1,1,-0.504713,0.645569,0.341329,0.071254,0.316608,0.232325,0.003045,negative,type surgery
1,1,2,0.000000,0.638464,0.159982,0.529674,0.094432,0.052398,0.026757,neutral,regular eye doctor
2,1,3,0.000000,0.629403,0.263028,0.275554,0.262074,0.081493,0.048659,neutral,eye doctor
3,1,4,0.965996,0.603728,0.279632,0.114120,0.182594,0.116771,0.051658,positive,aesthetic appearance eye
4,1,5,0.965996,0.573643,0.167237,0.594232,0.061398,0.018594,0.017416,positive,vision


In [56]:
# Re displaying posts for analysis
dfptposts.head()

,users_id,username,id,threads_id,number,content,date,sent_score,label
0,4,"Sandra M Brown, MD",4,2,1,\n Go to www.aao.org and search for...,2009-11-11 19:42:07,0.000000,neutral
1,4,"Sandra M Brown, MD",6,3,1,\n Your son should be evaluated by ...,2010-03-30 17:07:23,0.860021,positive
2,4,"Sandra M Brown, MD",8,4,1,\n Is it on your eyeBALL or on your...,2010-01-28 08:17:25,-0.864718,negative
3,4,"Sandra M Brown, MD",10,5,1,\n You have ptosis. Obviously ptosi...,2009-11-05 17:03:54,-0.760373,negative
4,4,"Sandra M Brown, MD",12,6,1,\n It sounds like the muscle that m...,2010-09-27 15:34:02,-0.323683,negative


In [57]:
# Join posts, keywords and scores
dfptkw=pd.merge(left=dfptposts[['id','users_id']], right=dfkwsent, how='left', left_on='id', right_on='posts_id')
dfptkw.drop(['id', 'users_id'], axis=1, inplace=True)

In [58]:
# All keyword appearances in posts
len(dfptkw)

758

In [59]:
dfptkw.head()

,posts_id,keywords_id,sent_score,relevance,sadness_score,joy_score,fear_score,disgust_score,anger_score,label,kwclean
0,4.0,31.0,0.0000,0.663188,0.205818,0.056872,0.370259,0.059339,0.110661,neutral,doctor
1,4.0,32.0,0.0000,0.623630,0.205818,0.056872,0.370259,0.059339,0.110661,neutral,wwwaaoorg
2,4.0,13.0,0.0000,0.613217,0.205818,0.056872,0.370259,0.059339,0.110661,neutral,specialist
3,4.0,33.0,0.0000,0.456296,0.205818,0.056872,0.370259,0.059339,0.110661,neutral,oculoplastics
4,6.0,41.0,0.8797,0.927342,0.015091,0.080987,0.104112,0.056071,0.022565,positive,eye md


In [60]:
# Keyword count and scores aggregation
gb = dfptkw.groupby('kwclean')
# INSERT THE SENT/ EMOTION SCORE TO ANALYZE 
# (i.e. sent_score, anger_score, disgust_score, fear_score, joy_score or sadness_socre)
dfsentcalcs=gb['sent_score'].agg([len, np.mean, np.std]).sort_values(by='len', ascending=False)
dfsentcalcs=dfsentcalcs.reset_index()
gb2=dfptkw.groupby(['kwclean', 'label'])
dfsentcat=gb2['label'].agg(len)
dfsentcat=dfsentcat.unstack(level=-1)
dfsentcat=dfsentcat.reset_index() 
dfsentanalysis=pd.merge(left=dfsentcat, right=dfsentcalcs, how='inner', left_on='kwclean', right_on='kwclean').sort_values('len', ascending=False)

In [61]:
# Total number of processed keywords
len(dfsentanalysis)

500

In [62]:
dfsentanalysis.head(250)

,kwclean,negative,neutral,positive,len,mean,std
296,oculoplastics,2.0,12.0,6.0,20.0,0.163794,0.446476
158,eye,14.0,2.0,3.0,19.0,-0.424511,0.576918
307,ophthalmologist,4.0,10.0,1.0,15.0,-0.109299,0.324501
496,wwwaaoorg,NaN,12.0,1.0,13.0,0.055646,0.200635
238,jch md,NaN,12.0,NaN,12.0,0.000000,0.000000
...,...,...,...,...,...,...,...
462,tumor,1.0,NaN,NaN,1.0,-0.858893,NaN
354,procedure result,1.0,NaN,NaN,1.0,-0.904469,NaN
433,tear drainage canal,1.0,NaN,NaN,1.0,-0.330709,NaN
370,referral,1.0,NaN,NaN,1.0,-0.675716,NaN


In [63]:
# INSERT YOUR CSV NAME HERE
dfsentanalysis.to_csv('<YOUR CSV NAME>')